# 줄리아를 생각하다(Think julia)

## Chapter 13. 사례 연구: 자료구조 선택하기

사례 연구로 자료구조 선택 시 고려할 사항들을 알아보고 실습하는 챕터

### 13.1 단어 빈도 분석

* **연습 13-1** 파일을 읽어서 각 줄을 단어들로 나누고, 공백문자와 문장부호를 제거한 후 소문자로 변환하는 프로그램을 작성

* **연습 13-2** 구텐베르크 프로젝트에서 저작권이 만료된 책을 다운로드, 책에 나오는 단어의 개수와 각 단어의 빈도 확인

* **연습 13-3** 상위 빈도 20개 단어 출력

* **연습 13-4** 단어 목록(사전)에 없는 것을 모두 출력, 오타 체크, 추가 목록 개수 확인 등

### 13.2 난수

* 대부분의 컴퓨터 프로그램은 같은 입력을 주면, 매번 같은 출력을 함 -> 결정론적(deterministic) 

* 비결정론적으로 동작하게 만드는 방법 -> **의사난수(pseudorandom number)** 생성 알고리즘

    * 결정론적 연산에 의해 생성되기 때문에 진짜 난수는 아님   
    
   
*  **rand 함수** : 0.0 이상 1.0 미만의 무작위 부동소수점 수 반환

In [1]:
# rand 함수 사용 예

for i in 1:10
    x = rand()
    println(x)
end

0.9745007219504291
0.5258988384247912
0.5048351182518076
0.01531547290925206
0.8569725121319413
0.8909916225065069
0.4546419559520034
0.6248474381441199
0.7303189491681726
0.5084298276632762


In [2]:
# rand 함수에 반복자나 배열을 인수로 주면 무작위로 고른 원소를 반환

for i in 1:10
    x = rand(1:6)
    print(x, " ")
end

6 1 5 4 6 6 4 5 1 1 

### 13.3 단어 히스토그램

제인 오스틴의 소설 '엠마'를 읽어서, 파일 안에 있는 단어에 대한 히스토그램을 만드는 프로그램 작성

* processfile 함수는 루프를 돌며 파일에서 한 줄씩 읽어 processline 함수에 전달

* processline은 replace 함수를 사용해 하이픈을 공백으로 치환하고 split 함수를 이용해 line을 문자열의 배열로 조각냄

* 단어의 배열을 순회하며 filter, isletter, lowercase를 이용해 문장부호를 제거하고 소문자로 반환함

* 최종적으로 processline에서 새로운 항목이나 기존 항목 값을 증가시킴으로써 히스토그램을 갱신함

* 파일에 있는 전체 단어 숫자를 카운트 하는 totalwords 함수 작성

* 중복을 제거한 단어의 숫자를 카운트 하는 differentwords 함수 작성

In [3]:
function processfile(filename)
    hist = Dict()
    for line in eachline(filename)
        processline(line, hist)
    end
    hist
end;

function processline(line, hist)
    line = replace(line, '-' => ' ')
    for word in split(line)
        word = string(filter(isletter, [word...])...)
        word = lowercase(word)
        hist[word] = get!(hist, word, 0) + 1
    end
end;

hist = processfile("./data/emma.txt");

In [4]:
function totalwords(hist)
    sum(values(hist))
end

totalwords (generic function with 1 method)

In [5]:
function differentwords(hist)
    length(hist)
end

differentwords (generic function with 1 method)

In [6]:
# 결과 출력

println("Total number of words: ", totalwords(hist), "\n")

println("Number of different words: ", differentwords(hist), "\n")

Total number of words: 162742

Number of different words: 7380



### 13.4 가장 흔한 단어들

가장 흔하게 사용된 단어를 찾기 위해서는 단어와 빈도를 원소로 하는 튜플의 배열을 만든 후, 배열을 정렬

In [7]:
function mostcommon(hist)
    t = []
    for (key, value) in hist
        push!(t, (value, key))
    end
    reverse(sort(t))
end

mostcommon (generic function with 1 method)

In [8]:
t = mostcommon(hist)

println("The most common words are: ")
for (freq, word) in t[1:10]
    println(word, "\t", freq) 
end

The most common words are: 
to	5295
the	5266
and	4931
of	4339
i	3191
a	3155
it	2546
her	2483
was	2400
she	2364


### 13.5 선택적 매개변수

단어의 빈도를 선택할 수 있도록 매개변수 조정 (num 인수 추가, 기본값은 10으로)

In [9]:
function printmostcommon(hist, num = 10)
    t = mostcommon(hist)
    println("The most common words are: ")
    for (freq, word) in t[1:num]
        println(word, "\t", freq)
    end
end

printmostcommon (generic function with 2 methods)

In [10]:
printmostcommon(hist)

The most common words are: 
to	5295
the	5266
and	4931
of	4339
i	3191
a	3155
it	2546
her	2483
was	2400
she	2364


In [11]:
printmostcommon(hist, 20)

The most common words are: 
to	5295
the	5266
and	4931
of	4339
i	3191
a	3155
it	2546
her	2483
was	2400
she	2364
in	2199
not	2161
you	2053
be	1987
he	1811
that	1809
had	1626
but	1446
as	1443
for	1371


* 선택적 인수는 디폴트 값에 우선하며, 일반적으로 필수 매개변수가 앞에 있고 선택적 매개변수가 뒤에 있음

### 13.6 딕셔너리에 대한 차집합

* subtract 함수는 딕셔너리 d1과 d2를 받아, d1에는 있고 d2에는 없는 모든 키를 원소로 하는 새로운 딕셔너리를 반환함

In [12]:
function subtract(d1, d2)
    res = Dict()
    for key in keys(d1)
        if key ∉ keys(d2)
            res[key] = nothing # 값은 신경쓰지 않기 위해 nothing으로 처리
        end
    end
    res
end

subtract (generic function with 1 method)

* 내려받은 책에는 나오지만 words.txt 에는 없는 단어를 찾으려면, 다음과 같이 processfile 함수로 words.txt의 히스토그램을 만든 후 차집합을 구함

In [15]:
words = processfile("./data/words.txt")
diff = subtract(hist, words)

println("Words in the book that aren't in the word list: ")
for word in keys(diff)
    print(word, " ")
end

Words in the book that aren't in the word list: 
outree quicksighted outwardly adelaide jeffereys unreserved dixons betweens groundless jamess westons remembrance unsuccessfully hawkinss rencontre deservedly favourably incommoded unfavourable wiltshire bateses recollecting newsletters recollected irresistibly pianoforte undiscerned ungraciously humourist bitnet placidity swisserland il xxxxxxxxx internet prosings constitutions ult christian xvi illinois outstepped november woodhouse experienced favourite neptune ocr wallises companionably tremblings recollect uninterruptedly ing emma smallridges disingenuousness injustice admirably appellation familiarise hartuiucvmd surry ix c ii xiii cromer shakespeare vii unconvinced coxs tranquillised complimenter broadwood bellas goodnatured dont richardson doatingly braithwaites unimpeded michaelmas reanimation endthe neighbourhood william influenced harriets disclaimer churchill twelvemonth tunbridge england gentlemans imaginist unexampled hodge

### 13.7 무작위 단어

무작위 단어를 고르는 가장 간단한 알고리즘 구현

* 빈도만큼 중복해서 단어가 들어 있는 배열을 만든 후, 그 배열에서 단어를 고르는 것

In [16]:
function randomword(h)
    t = []
    for (word, freq) in h
        for i in 1:freq
            push!(t, word)
        end
    end
    rand(t)
end

randomword (generic function with 1 method)

In [17]:
randomword(hist)

"speaking"

### 13.8 용어집

* **결정론적(deterministic)**

    - 어떤 프로그램이 동일한 입력에 대해, 항상 동일한 동작을 수행함을 일컫음
```
```
* **의사 무작위(pseudoramdom)**

    - 사실은 결정론적으로 생성되나 겉으로는 무작위처럼 보이는 것을 일컫음
```
```
* **디폴트 값(default value)**

    - 인수가 주어지지 않을 때, 선택적 매개변수에 할당되는 기본값
```
```
* **우선하기(to override)**

    - 디폴트 값을 주어진 인수로 덮어 씌우는행위
```
```
* **벤치마킹(benchmarking)**

    - 자료구조를 선택하는 과정으로 여러 대안을 구현해서 샘플 입력으로 테스트하고 비교하는 것
```
```
* **러버덕킹(rubberducking)**

    - 러버덕 같은 무생무에게 문제를 설명함으로써 디버깅하는 방법. 러버덕에게라도 정확히 설명하려고 하다 보면 문제 해결에 도움이 됨